In [2]:
import nibabel as nib
import numpy as np
import matplotlib.pylab as plt
import cvxpy as cp
import pandas as pd
from os.path import join
from scipy.spatial.distance import cdist
from scipy.sparse import csgraph
from scipy.stats import skew, kurtosis
from itertools import product
from sklearn.model_selection import KFold
from matplotlib.animation import FuncAnimation
from nilearn import masking
from nilearn.image import new_img_like
from nilearn.plotting import plot_stat_map, view_img
import os, sys
from config import Config
from pathlib import Path
from nilearn.image import mean_img
from nilearn.image import resample_to_img
import warnings

In [3]:
ses = 1
sub = '04'
runs_to_process = [1, 2]

anat_img = nib.load(f'/mnt/TeamShare/Data_Masterfile/H20-00572_All-Dressed/PRECISIONSTIM_PD_Data_Results/fMRI_preprocessed_data/Rev_pipeline/derivatives/sub-pd0{sub}/ses-{ses}/anat/sub-pd0{sub}_ses-{ses}_T1w_brain_2mm.nii.gz')
anat_img = nib.as_closest_canonical(anat_img) #????

base_path = '/mnt/TeamShare/Data_Masterfile/H20-00572_All-Dressed/PRECISIONSTIM_PD_Data_Results/fMRI_preprocessed_data/Rev_pipeline/derivatives'

fmt = lambda x: f"{x:.3f}"
fmt_arr = lambda a: np.array2string(np.asarray(a), precision=3, separator=', ', suppress_small=True)

In [4]:
def remove_background(mask, anat_file, bold_data):
    mask_in_run1_img = resample_to_img(source_img=mask, target_img=anat_file, interpolation='nearest', force_resample=True, copy_header=True)
    anat_data = anat_file.get_fdata()
    valid_run1 = np.isfinite(anat_data) & (anat_data != 0)
    mask_in_run1 = mask_in_run1_img.get_fdata().astype(bool)
    combined_mask = mask_in_run1 & valid_run1
    mask_4d = combined_mask[..., np.newaxis]

    bold_data_masked = bold_data * mask_4d
    active_voxels = np.any(bold_data_masked != 0, axis=-1)
    final_mask = active_voxels & combined_mask
    masked_timeseries_3d = np.where(final_mask[..., None], bold_data, 0)

    active_voxels = np.any(masked_timeseries_3d != 0, axis=-1)
    coords = np.array(np.where(active_voxels)).T
    xmin, ymin, zmin = coords.min(axis=0)
    xmax, ymax, zmax = coords.max(axis=0)

    masked_timeseries_3d_cropped = masked_timeseries_3d[xmin:xmax+1, ymin:ymax+1, zmin:zmax+1, :]
    return masked_timeseries_3d_cropped, xmin, ymin, zmin

def save_beta_html(beta_3d, anat_img, bold_img, A, title, fname, vmin_all, vmax_all):
    beta_on_anat = nib.Nifti1Image(beta_3d, A, bold_img.header)
    view = view_img(
        beta_on_anat,
        bg_img=anat_img,
        threshold=None,  
        symmetric_cmap=False,
        cmap='jet',
        # vmin=vmin_all, vmax=vmax_all,
        title=title,
        colorbar=True,
        cut_coords=(0,0,0)
    )
    view.save_as_html(file_name=fname)

def robust_vrange(img, lo=2, hi=98):
    """Percentile-based vmin/vmax on plain ndarray (no masked array)."""
    data = np.asarray(img.get_fdata(), dtype=float)
    data = data[np.isfinite(data)]
    if data.size == 0:
        return 0.0, 1.0
    vmin, vmax = np.percentile(data, [lo, hi])
    return float(vmin), float(vmax)

GLM Overview

In [5]:
glm_dict = np.load('/home/zkavian/thesis_code_git/GLMOutputs-sub04-ses1/TYPED_FITHRF_GLMDENOISE_RR.npy', allow_pickle=True).item()

print("Keys:", list(glm_dict.keys()))
print("betasmd shape:", glm_dict['betasmd'].shape, "\n")

# ---------- Noisepool percentage ----------
noisepool = glm_dict['noisepool'].astype(np.uint8)
coord = np.where(noisepool)
num_voxels = noisepool.size
pct_noise = len(coord[0]) / num_voxels * 100
print(f"Detected Noise Voxels: {fmt(pct_noise)} %")

# ---------- FRAC value summaries ----------
def unique_safely(x):
    if np.ma.isMaskedArray(x):
        return np.unique(x.compressed())
    return np.unique(x)

FRAC = glm_dict['FRACvalue']
all_vals = unique_safely(FRAC)
noisepool_vals = unique_safely(FRAC[coord])

print("Fractional Regularization Level:")
print("  All voxels unique:", fmt_arr(all_vals))
print("  Noisepool voxels unique:", fmt_arr(noisepool_vals))
print()

# ---------- Per-run splits from GLM (shared) ----------
beta = glm_dict['betasmd']
beta_run1, beta_run2 = beta[..., :90], beta[..., 90:]
R2_run1, R2_run2 = glm_dict['R2run'][:,:,:,0], glm_dict['R2run'][:,:,:,1]

def stats_str(x):
    return f"max {fmt(np.nanmax(x))}, min {fmt(np.nanmin(x))}, mean {fmt(np.nanmean(x))}"

print("Beta Value:")
print("  Run1:", stats_str(beta_run1))
print("  Run2:", stats_str(beta_run2))
print()
print("R2 Value:")
print("  Run1:", stats_str(R2_run1))
print("  Run2:", stats_str(R2_run2))
print()

# ---------- % non-NaN (active) voxels ----------
n_total = beta_run1.size
pct_active_run1 = np.count_nonzero(~np.isnan(beta_run1)) / n_total * 100
pct_active_run2 = np.count_nonzero(~np.isnan(beta_run2)) / n_total * 100
print(f"Non-NaN (active) voxels: Run1 {fmt(pct_active_run1)} %, Run2 {fmt(pct_active_run2)} %")

# Beta means (shared computation; used per run below)
warnings.filterwarnings("ignore", message="Mean of empty slice", category=RuntimeWarning)
beta_mean_run1 = np.nanmean(beta_run1, axis=-1)
beta_mean_run2 = np.nanmean(beta_run2, axis=-1)

# Consistent color scaling across both runs
both_beta_vals = np.concatenate([np.ravel(beta_mean_run1), np.ravel(beta_mean_run2)])
vmax_all = np.nanpercentile(np.abs(both_beta_vals), 99)
vmin_all = -vmax_all

Keys: ['HRFindex', 'HRFindexrun', 'glmbadness', 'pcvoxels', 'pcnum', 'xvaltrend', 'noisepool', 'pcregressors', 'betasmd', 'R2', 'R2run', 'rrbadness', 'FRACvalue', 'scaleoffset', 'meanvol']
betasmd shape: (69, 92, 73, 180) 

Detected Noise Voxels: 29.452 %
Fractional Regularization Level:
  All voxels unique: [0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55, 0.6 ,
 0.65, 0.7 , 0.75, 1.  ]
  Noisepool voxels unique: [0.05, 0.1 , 0.15, 0.2 ]

Beta Value:
  Run1: max 12023.069, min -2978.961, mean 0.009
  Run2: max 2439.105, min -4227.346, mean 0.026

R2 Value:
  Run1: max 37.737, min -6.138, mean 4.646
  Run2: max 61.058, min -21.858, mean 5.038

Non-NaN (active) voxels: Run1 45.410 %, Run2 45.410 %


BOLD Image and Masked BOLD Image on the Brain Anatomy

In [6]:
warnings.filterwarnings("ignore", message=".*'partition' will ignore the 'mask' of the MaskedArray.*", category=UserWarning, module=r"numpy\._core\.fromnumeric")

for run in runs_to_process:
    print(f"\n===== Processing run {run} =====")

    # Load BOLD for this run
    data_name = f'sub-pd0{sub}_ses-{ses}_run-{run}_task-mv_bold_corrected_smoothed_reg_2mm.nii.gz'
    BOLD_path_org = join(base_path, f'sub-pd0{sub}', f'ses-{ses}', 'func', data_name)
    bold_img = nib.load(BOLD_path_org)
    func_img = nib.as_closest_canonical(bold_img)
    bold_data = bold_img.get_fdata()
    print("Raw BOLD Data Shape: ", bold_data.shape)

    # Quick overlay of mean BOLD on anatomy (saved)
    mean_bold = mean_img(bold_img, copy_header=True) 
    vmin, vmax = robust_vrange(mean_bold)
    view = view_img(mean_bold, bg_img=anat_img, title='BOLD Image on Anatomic Brain', threshold=None, symmetric_cmap=False, vmin=vmin, vmax=vmax, dim=0)
    view.save_as_html(f"BOLD_data_on_Anat_sub{sub}_ses{ses}_run{run}.html")

    # Mask path (generalized)
    mask_path = f'/mnt/TeamShare/Data_Masterfile/H20-00572_All-Dressed/PRECISIONSTIM_PD_Data_Results/fMRI_preprocessed_data/Rev_pipeline/derivatives/sub-pd0{sub}/ses-{ses}/anat/sub-pd0{sub}_ses-{ses}_T1w_brain_mask.nii.gz'
    mask = nib.load(mask_path)

    # Remove background and crop
    bold_data_masked, xmin, ymin, zmin = remove_background(mask, anat_img, bold_data)
    print("Masked shape:", bold_data_masked.shape)

    # Adjust affine to account for cropping
    A = bold_img.affine.copy()
    A[:3, 3] = A[:3, 3] + A[:3, :3] @ np.array([xmin, ymin, zmin], dtype=float)
    bold_data_masked_img = nib.Nifti1Image(bold_data_masked, A, bold_img.header)

    # Overlay masked mean BOLD on anatomy (saved)
    mean_bold_masked = mean_img(bold_data_masked_img, copy_header=True)  # fixes FutureWarning
    vmin_m, vmax_m = robust_vrange(mean_bold_masked)
    view = view_img(mean_bold_masked, bg_img=anat_img, title='Masked BOLD Image on Anatomic Brain')
    view.save_as_html(f"Masked_BOLD_data_on_Anat_sub{sub}_ses{ses}_run{run}.html")

    # Noisepool overlay (per run file)
    noisepool_img = nib.Nifti1Image(noisepool, A, bold_img.header)
    vmax_np = float(np.nanmax(np.asarray(noisepool, dtype=float)))
    view = view_img(noisepool_img, bg_img=anat_img, threshold=0.5, title='Noisepool voxels', resampling_interpolation='nearest', 
                    symmetric_cmap=False, vmin=0.0, vmax=vmax_np)
    view.save_as_html(file_name=f"Noisepool_Mask_on_the_brain_sub{sub}_ses{ses}_run{run}.html")



===== Processing run 1 =====
Raw BOLD Data Shape:  (90, 128, 85, 850)
Masked shape: (69, 92, 73, 850)

===== Processing run 2 =====
Raw BOLD Data Shape:  (90, 128, 85, 850)
Masked shape: (69, 92, 73, 850)


Plot Beta Values on the Brain

In [ ]:
for run in runs_to_process:
    # Select this run's beta/R2
    if run == 1:
        beta_mean_this = beta_mean_run1
        R2_this = R2_run1
    else:
        beta_mean_this = beta_mean_run2
        R2_this = R2_run2

    # Per-run beta maps (all voxels)
    save_beta_html(beta_mean_this, anat_img, A, f"All Beta", f"Beta_all_voxels_sub{sub}_ses{ses}_run{run}.html")
    save_beta_html(np.abs(beta_mean_this), anat_img, A, f"All Abs Beta", f"Beta_abs_all_voxels_sub{sub}_ses{ses}_run{run}.html")

    # Histogram & binning based on R2 for THIS run (saved)
    bin_width = 10
    X = R2_this.ravel()
    bins = np.arange(np.nanmin(X), np.nanmax(X) + bin_width, bin_width)

    plt.figure(figsize=(8,8))
    n, bins, patches = plt.hist(X, bins=bins)
    plt.tight_layout()
    plt.savefig(f"R2_hist_sub{sub}_ses{ses}_run{run}.png", dpi=150)
    plt.close()

    print(f"Histogram Bins (run {run}): {bins}, Number of bins: {int(len(patches))}")

    # Make per-bin beta masks & save
    n_bins = int(len(patches))
    for i in range(n_bins):
        low, high = float(bins[i]), float(bins[i+1])
        if i < n_bins - 1:
            mask_i = (R2_this >= low) & (R2_this < high)
        else:
            mask_i = (R2_this >= low) & (R2_this <= high)

        mask_beta_3d = np.full(beta_mean_this.shape, np.nan, dtype=float)
        mask_beta_3d[mask_i] = beta_mean_this[mask_i]
        c_min, c_max = np.nanmin(mask_beta_3d), np.nanmax(mask_beta_3d)
        print(f"run {run} bin {i+1}: min {c_min}, max {c_max}")

        beta_on_anat = nib.Nifti1Image(mask_beta_3d, A, bold_img.header)
        view = view_img(
            beta_on_anat,
            bg_img=anat_img,
            title=f'bin {i+1},  [{fmt(low)},{fmt(high)}]',
            colorbar=True,
            cmap='seismic'
        )
        view.save_as_html(file_name=f'Beta_mask_bin{i+1}_sub{sub}_ses{ses}_run{run}.html')

    # Low/High R2 groups for THIS run (saved)
    r2_min = np.nanmin(R2_this)
    r2_max = np.nanmax(R2_this)
    r2_mean = np.nanmean(R2_this)

    low_mask = (R2_this >= r2_min) & (R2_this < r2_mean)
    high_mask = (R2_this >= r2_mean) & (R2_this <= r2_max)

    low_masked_beta = beta_mean_this[low_mask]
    high_masked_beta = beta_mean_this[high_mask]
    num_voxels_non_nan = R2_this[~np.isnan(R2_this)].size

    print(f"Low Mask Group (run {run}): [{fmt(r2_min)}, {fmt(r2_mean)}]: {fmt(len(low_masked_beta)/num_voxels_non_nan*100)}% of non-nan Voxels")
    print(f"High Mask Group (run {run}): [{fmt(r2_mean)}, {fmt(r2_max)}]: {fmt(len(high_masked_beta)/num_voxels_non_nan*100)}% of non-nan Voxels")

    low_beta_3d = np.full(beta_mean_this.shape, np.nan, dtype=float)
    low_beta_3d[low_mask] = beta_mean_this[low_mask]
    high_beta_3d = np.full(beta_mean_this.shape, np.nan, dtype=float)
    high_beta_3d[high_mask] = beta_mean_this[high_mask]

    beta_on_anat = nib.Nifti1Image(low_beta_3d, A, bold_img.header)
    view = view_img(beta_on_anat, bg_img=anat_img, title=f'Low R2 Beta (Run {run})', colorbar=True, cut_coords=(0,0,0))
    view.save_as_html(file_name=f'Beta_low_mask_sub{sub}_ses{ses}_run{run}.html')

    beta_on_anat = nib.Nifti1Image(high_beta_3d, A, bold_img.header)
    view = view_img(beta_on_anat, bg_img=anat_img, title=f'High R2 Beta (Run {run})', colorbar=True, cut_coords=(0,0,0))
    view.save_as_html(file_name=f'Beta_high_mask_sub{sub}_ses{ses}_run{run}.html')

# %%
print("\nAll done. Per-run HTML files and histograms have been saved in the current directory.")
